In [42]:
import random

In [43]:
# Генерация кривой
def GenerateEC():
    #p = random.randint(10**18, 10**25)
    while (true):
        p = random.randint(10**15, 10**20)
        if p in Primes():
            break
    #p = 11
    K = GF(p)
    A = K(randint(1,p))
    #A = 4
    B = K(randint(1,p))
    #B = 10
    # генерация кривой над конечным полем K
    return K, p, A, B
# Алгоритм Шуфа
def Schf(E, p):
    ti = []
    li, m = ListLi(p)
    print('l_i - {}'.format(li))
    # вычисление значений для сравнений по модулю l_i
    for i in li:
        f = FrobeniusTrace(E, i, p) # след Фробениуса по mod l
        ti.append(f)
    print('T_i - {}'.format(ti))
    # КТО 
    T = crt(ti, li)
    return T, m
# Поиск простых чисел Li
def ListLi(p):
    li = []
    m = 1
    fs = 4 * sqrt(p)
    for i in Primes():
        m = m * i # по т. Хассе
        if m <= fs:
            li.append(i)
        else:
            break
    li.append(i)
    return li, m  
# след Фробениуса по mod l
def FrobeniusTrace(E, l, p):
    PR.<t> = PolynomialRing(E.base_field()) # расширяем тк случай 3 (не 2 и 3)
    A = E.a4()
    B = E.a6()
    
    if l != 2:
        h = E.division_polynomial(l, t) 
        K.<x> = QuotientRing(PR, ideal(h)) # построение факторкольца F_p[x]/(h(x)) по идеалу полинома деления h(x)
        f = x^3 + A*x + B
        # вычисляем точку 𝜋
        xp = K(x**p)
        yp = K(f**((p-1) // 2))
        # вычисляем точку 𝜋^(2)
        xp_2, yp2 = xp2_yp2(xp, yp, K)     
        # вычисляем точку p_l
        pl_x, pl_y = Multiply(K(x), K(1), p, l, K, A, f)
        # вычисляем g = 𝜋^(2) + 𝑝 
        g_x, g_y = Sum(xp_2, yp2, pl_x, pl_y, K, A, f)
        if g_y == 'Error':
            print('error h', l)
            h = gcd(h, lift(g_x)) # находим НОД lift - возвращает полином в кольце
            K.<x> = QuotientRing(PR, ideal(h))
            xp_2, yp2 = xp2_yp2(xp, yp, K)
            # вычисляем точку p_l
            pl_x, pl_y = Multiply(K(x), K(1), p, l, K, A, f)
            # вычисляем g
            g_x, g_y = Sum(xp_2, yp2, pl_x, pl_y, K, A, f)
            if g_y == 'Error':
                print('err h kk', l)

        # проверяем g
        if g_x == 0 or g_y == 0:
            return 0
        elif g_x == xp and g_y == yp:
            return 1
        elif g_x == (-1)*xp and g_y == (-1)*yp:
            return -1
        else:
            k = 2
            kxp, kyp = Multiply(xp, yp, k, l, K, A, f)
            if g_x == kxp and  g_y == kyp:
                    return k
              
            k += 1
            while k < l:
                kxp, kyp = Multiply(xp, yp, k, l, K, A, f)
                if g_x == kxp:
                    if g_y == kyp:
                        return k
                k += 1
        return 0
    else:
        f = t**3 + A*t + B
        if f.is_irreducible(): # проверка является ли этот многочлен неприводимым над F2
            return 1
        else:
            return 0
# Вычисляем  квадрат эндоморфизма Фрабениуса 
def xp2_yp2(a, b, RR):
    xp_2 = Comp(a, a, RR)
    yp_2 = Comp(b, a, RR)
    yp_2 = RR(yp_2 * b)
    return xp_2, yp_2 

def Comp(a, b, RR):
    li = a.list()
    res = 0
    d = 0
    for i in li:
        res = RR(res + i * (b^d))
        d = d + 1
    return res

def Multiply(a, b, p, l, RR, A, f):
    d = p % l
    if d == 0:
        return 0, 0
    elif d == 1:
        return a, b
    elif d == 2:
        return Sum(a, b, a, b, RR, A, f)
    else:
        a0 = a
        b0 = b
        mask = bin(d)[3:]
        for i in range(len(mask)):
            a0, b0 = Sum(a0, b0, a0, b0, RR, A, f)
            if mask[i] == '1':
                a0, b0 = Sum(a0, b0, a, b, RR, A, f)
        return a0, b0
    
def Sum(a1, b1, a2, b2, RR, A, f):
    if a1 == a2 and b1 == b2:
        try:
            m = RR((3 * a1**2 + A) / (2 * b1 * f))
        except ZeroDivisionError:
            if RR(2 * b2 * f) == 0:
                return 1, 0  
            else:
                div = RR(2 * b1 * f)
                return div, 'Error'
    else:
        try:
            m = RR((b2 - b1) / (a2 - a1))
        except ZeroDivisionError:
            if RR(a2 - a1) == 0:
                return 1, 0
            div = RR(a2 - a1)
            return div, 'Error'

    a3 = RR(f * m**2 - a1 - a2)
    b3 = RR(m * (a1 - a3) - b1)
    return a3, b3   

In [46]:

K, p, A, B = GenerateEC()
print(K)
print ('p = {}'.format(p), 'A = {}'.format(A), 'B = {}'.format(B))
E = EllipticCurve(K, [A,B])
print ('{}'.format(E))
T, Z = Schf(E, p)
if T > (Z // 2):
    T = T - Z
N = p + 1 - T 
if N == E.order():
    print ('T = {}'.format(T))
    print ('N = {}'.format(N))



Finite Field of size 87123521302319049661
p = 87123521302319049661 A = 16551104974697363300 B = 16062114590245429001
Elliptic Curve defined by y^2 = x^3 + 16551104974697363300*x + 16062114590245429001 over Finite Field of size 87123521302319049661
l_i - [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
error h 3
error h 7
T_i - [0, 1, 0, 5, 7, 0, 3, 7, 12, 2, 29]
T = -1934809760
N = 87123521304253859422


In [45]:
# для расширения
def Fcm(T,p):
    x = var('x')
    equation = x^2-T*x + p
    return equation, solve(equation,x)

p = 7
A = 2
B = 3
K = GF(p)
E = EllipticCurve(K, [A,B])
N,T = Schf(E, p)
print (f'Frobenius trace: T = {T}')
res = Fcm(T,p)
print (f'Roots - {res[1]}')

y = var('y')
print(solve([p**6 + 1 + (-sqrt(218) + 15)**6 + (sqrt(218) + 15)**6 == y],y)[0])



l_i - [2, 3, 5]
error h 3
T_i - [0, 2, 2]
Frobenius trace: T = 30
Roots - [
x == -sqrt(218) + 15,
x == sqrt(218) + 15
]
y == 695493864
